In [ ]:
%load_ext autoreload
%autoreload 2

import json
from tqdm.auto import tqdm
from pathlib import Path
import plotting
import networkx as nx 
from joblib import Parallel, delayed
import contextlib
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from utils import make_graph
import pandas as pd
from copy import deepcopy
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import hashlib

In [ ]:
# https://stackoverflow.com/questions/24983493/tracking-progress-of-joblib-parallel-execution
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback:
        def __init__(self, time, index, parallel):
            self.index = index
            self.parallel = parallel

        def __call__(self, index):
            tqdm_object.update()
            if self.parallel._original_iterator is not None:
                self.parallel.dispatch_next()

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close() 

In [ ]:
logs_path = Path('train_logs_single_run')
logs_paths = list(logs_path.glob("*.json"))

logs_ok = []
logs_not_ok = []
for idx, log_path in tqdm(enumerate(logs_paths), total=len(logs_paths)):
    with open(log_path, "r") as f:
        log = json.load(f)
    recepie = json.loads(log['recepie'])
    log['recepie'] = recepie
    log['idx'] = idx
    
    if log['status'] == 'OK':
        logs_ok.append(log)
    else:
        logs_not_ok.append(log)

In [ ]:
print("number ok: ", len(logs_ok))
print("number not ok: ", len(logs_not_ok))

In [ ]:
len(logs_not_ok)/len(logs_ok)

In [ ]:
def map_node(x):
    for v in ['node', 'h_prev', 'h_new']:
        if x.find(v) != -1:
            x = v
    if x not in ['x', 'node', 'h_prev', 'h_new']: # to make lstm and gru recepies standard
        x = 'node'
    return x

In [ ]:
def make_graph_2(recepie):
    
    G = nx.DiGraph()
    for n in recepie.keys():
        if n not in G.nodes():
            G.add_node(n)
        for k in recepie[n]['input']:
            if k not in G.nodes():
                G.add_node(k)
            G.add_edge(n, k, label=recepie[n]['op'])
            G.add_edge(k, n, label='rev_' + recepie[n]['op'])
    return G

In [ ]:
def random_walk_features(G, steps=10):
    walk = []
    node = np.random.choice(G.nodes(), 1)[0]
    for _ in range(steps):
        k = np.random.choice(list(G.adj[node]), 1)[0]
        walk.extend([map_node(node), G.adj[node][k]['label']])
        node = k
    walk.append(map_node(node))
    return walk

In [ ]:
def make_graph_sentences(G, sentences_num=20):
    sentences = []
    for _ in range(sentences_num):
        sentences.extend(random_walk_features(G) + ['.'])
    return sentences

In [ ]:
def feature_extractor(log):
    name = f"log_{log['idx']}"
    recepie = log['recepie']
    G = make_graph_2(recepie)
    doc = TaggedDocument(words=make_graph_sentences(G), tags=[name])
    return doc

In [ ]:
document_collections = Parallel(n_jobs=-2)(delayed(feature_extractor)(log) for log in tqdm(logs_not_ok+logs_ok))

In [ ]:
# takes a while!!
size = 10
# size = 50
doc2vec_model = Doc2Vec(document_collections, 
                        size=size, window=3, dm=1, min_count=0, workers=8, epochs=100, hs=1,
                        dbow_words=0)

In [ ]:
all_words = set()
for d in document_collections:
    all_words |= set(d.words)
len(all_words)

In [ ]:
not_dublicates_ok_logs = []
not_dublicates_not_ok_logs = []
not_dublicates_recepies = []
for log in logs_not_ok+logs_ok:
    recepie = log['recepie']
    if recepie not in not_dublicates_recepies:
        not_dublicates_recepies.append(recepie)
        if log['status'] == 'OK':
            not_dublicates_ok_logs.append(log)
        else:
            not_dublicates_not_ok_logs.append(log)
print("total: ", len(logs_not_ok+logs_ok))
print("without dublicates: ", len(not_dublicates_ok_logs+not_dublicates_not_ok_logs))

In [ ]:
#dump vectors
recepie_id_vectors = {log['recepie_id']:doc2vec_model.docvecs[f"log_{log['idx']}"]
                      for log in not_dublicates_ok_logs+not_dublicates_not_ok_logs}


In [ ]:
recepie_id_vectors_list = []
for k in recepie_id_vectors:
    k_dict = {'recepie_id':k}
    for i in range(doc2vec_model.vector_size):
        k_dict[f'v{i:02d}'] = recepie_id_vectors[k][i]
    recepie_id_vectors_list.append(k_dict)

In [ ]:
df_recepie_vectors = pd.DataFrame(recepie_id_vectors_list)

In [ ]:
df_recepie_vectors.head()

In [ ]:
# if size == 50:
#     df_recepie_vectors.to_csv('data/doc2vec_features.csv', index=False)
# elif size == 10:
#     df_recepie_vectors.to_csv('data/doc2vec_features_lowdim.csv', index=False)

In [ ]:
df_recepie_vectors = pd.read_csv('data/doc2vec_features.csv').set_index('recepie_id')

In [ ]:
trainset_list = [
    (df_recepie_vectors.loc[log['recepie_id']], int(log['status'] == 'OK'))
    for log in not_dublicates_ok_logs+not_dublicates_not_ok_logs
]

random.shuffle(trainset_list)

trainset_np, testset_np = np.array(trainset_list[:7000]), np.array(trainset_list[7000:])
train_X, train_y = np.array(list(trainset_np[:, 0])), np.array(list(trainset_np[:, 1]))
test_X, test_y = np.array(list(testset_np[:, 0])), np.array(list(testset_np[:, 1]))

print("Train:", len(trainset_np))
print("Test: ", len(testset_np))

num_train_not_ok = len(train_y) - train_y.sum()
print("\nTrain OK: ", train_y.sum())
print("Train not OK: ", num_train_not_ok)

num_test_not_ok = len(test_y) - test_y.sum()
print("\nTest OK: ", test_y.sum())
print("Test not OK: ", num_test_not_ok)

In [ ]:
tsne = TSNE(n_components=2, n_iter=300, verbose=True)

In [ ]:
E = tsne.fit_transform(train_X)

In [ ]:
plt.figure(figsize=(8, 8))
plt.scatter(E[:, 0], E[:, 1], s=3, c=train_y)

In [ ]:
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, precision_recall_curve, average_precision_score, roc_curve, auc 

In [ ]:
model = XGBClassifier(n_jobs=-1)
model.fit(train_X, train_y)

pred_y = model.predict_proba(test_X)
precision_xgboost, recall_xgboost, _ = precision_recall_curve(test_y, pred_y[:, 1])
fpr_xgboost, tpr_xgboost, _ = roc_curve(test_y, pred_y[:, 1])
roc_auc_axboost = auc(fpr_xgboost, tpr_xgboost)

f1_test_score = f1_score(test_y, np.argmax(pred_y, 1))
ap_test_score = average_precision_score(test_y, pred_y[:, 1])
print("XGBoost F1 score: ", f1_test_score)
print("XGBoost Average Precision score: ", ap_test_score)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression().fit(train_X, train_y)
pred_y = clf.predict_proba(test_X)
precision_lr, recall_lr, _ = precision_recall_curve(test_y, pred_y[:,1])
fpr_lr, tpr_lr, _ = roc_curve(test_y, pred_y[:, 1])
roc_auc_lr = auc(fpr_lr, tpr_lr)

f1_test_score = f1_score(test_y, np.argmax(pred_y, 1))
ap_test_score = average_precision_score(test_y, pred_y[:, 1])
print("Logistic Regression F1 score: ", f1_test_score)
print("CatBoost Average Precision score: ", ap_test_score)

In [ ]:
#plt.title('Receiver Operating Characteristic')
plt.figure(figsize=(6, 6))
plt.plot(fpr_xgboost, tpr_xgboost, label='XGBoost AUC = %0.2f' % roc_auc_axboost)
plt.plot(fpr_lr, tpr_lr, label='Logistic Regression AUC = %0.2f' % roc_auc_lr)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'k--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate', fontsize=16)
plt.xlabel('False Positive Rate', fontsize=16)

plt.savefig('data/figures/prediction_faulty.png', dpi=300, bbox_inches='tight')

In [ ]:
trainset_list = [
    (doc2vec_model.docvecs[f"log_{log['idx']}"], np.array(log['val_losses']).min())
    for log in not_dublicates_ok_logs
]


np.random.seed(0)
random.shuffle(trainset_list)

trainset_np, testset_np = np.array(trainset_list[:7000]), np.array(trainset_list[7000:])
train_X, train_y = np.array(list(trainset_np[:, 0])), np.array(list(trainset_np[:, 1]))
test_X, test_y = np.array(list(testset_np[:, 0])), np.array(list(testset_np[:, 1]))

print("Train:", len(trainset_np))
print("Test: ", len(testset_np))

num_train_not_ok = len(train_y) - train_y.sum()
print("\nTrain OK: ", train_y.sum())
print("Train not OK: ", num_train_not_ok)

num_test_not_ok = len(test_y) - test_y.sum()
print("\nTest OK: ", test_y.sum())
print("Test not OK: ", num_test_not_ok)

In [ ]:
plt.figure(figsize=(8, 8))
plt.scatter(E[:, 0], E[:, 1], s=3, c=train_y, cmap=plt.cm.plasma_r)
plt.colorbar()
plt.clim([4.5, 6])

In [ ]:
train_y

In [ ]:
(train_y > 6).mean()

In [ ]:
plt.figure(figsize=(8, 8))
plt.scatter(E[:, 0], E[:, 1], s=3, color='C0')
sub_inds = np.where(train_y > 6)[0]
plt.scatter(E[sub_inds, 0], E[sub_inds, 1], s=5, color='red')

In [ ]:
model = XGBClassifier(n_jobs=-1)
model.fit(train_X, train_y > 6)

pred_y = model.predict_proba(test_X)
precision_xgboost, recall_xgboost, _ = precision_recall_curve(test_y > 6, pred_y[:, 1])
fpr_xgboost, tpr_xgboost, _ = roc_curve(test_y > 6, pred_y[:, 1])
roc_auc_axboost = auc(fpr_xgboost, tpr_xgboost)

f1_test_score = f1_score(test_y > 6, np.argmax(pred_y, 1))
ap_test_score = average_precision_score(test_y > 6, pred_y[:, 1])
print("XGBoost F1 score: ", f1_test_score)
print("XGBoost Average Precision score: ", ap_test_score)

In [ ]:
#plt.title('Receiver Operating Characteristic')
plt.figure(figsize=(6, 6))
plt.plot(fpr_xgboost, tpr_xgboost, label='XGBoost AUC = %0.2f' % roc_auc_axboost)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'k--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate', fontsize=16)
plt.xlabel('False Positive Rate', fontsize=16)


In [ ]:
from xgboost import XGBRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import r2_score

regr = BaggingRegressor(XGBRegressor(n_estimators=100, max_depth=15), n_jobs=10, n_estimators=20, max_samples=0.5).fit(train_X, train_y)
regr_6 = BaggingRegressor(XGBRegressor(n_estimators=100, max_depth=15), n_jobs=10, n_estimators=20, max_samples=0.5).fit(train_X[train_y < 6], train_y[train_y < 6])
pred_y = regr.predict(test_X)
pred_y_6 = regr_6.predict(test_X)


In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(test_y, pred_y, s=1)
plt.xlabel('Testing log perplexity', fontsize=16)
plt.ylabel('Predicted testing log perplexity', fontsize=16)
plt.xlim([4.5, 7])
plt.ylim([4.5, 7])
plt.savefig('data/figures/prediction_loss.png', dpi=300, bbox_inches='tight')

In [ ]:
r2_score(test_y, pred_y)

In [ ]:
r2_score(test_y, pred_y_6)

In [ ]:
r2_score(test_y[test_y < 6], pred_y[test_y < 6])

In [ ]:
r2_score(test_y[test_y < 6], pred_y_6[test_y < 6])